In [1]:
# -*- encoding:urf-8 -*-
# Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Handle table like and matrices
import pandas as pd
import numpy as np

# Modeling Helper
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
import missingno as msno

# Configure visualization
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 8,6

train_path = '../input/train.csv'
test_path = '../input/test.csv'
hol_path = '../input/holiday.csv'
train_date_path = '../input/train_date.csv'
cache_path = '../input/cache/'
output_path = '../output/'
train_new_path = 'train_new_total.csv'
train_new = pd.read_csv(train_new_path)
train = pd.read_csv(train_path,encoding='gbk',engine='python')
test = pd.read_csv(test_path)
holiday = pd.read_csv(hol_path)

### 一天前的销量相关特征

In [49]:
coord = train_new.groupby(['Class','dayOfYear'],as_index=False)['saleCount'].agg({'saleCount':'mean'})
coord['last1DaySale'] = coord['saleCount'].shift(1)
coord['last1DaySale'].fillna(method='bfill',inplace=True)
coord['last2DaySale'] = coord['last1DaySale'].shift(1)
coord['last2DaySale'].fillna(method='bfill',inplace=True)
coord['last3DaySale'] = coord['last2DaySale'].shift(1)
coord['last3DaySale'].fillna(method='bfill',inplace=True)
coord['last4DaySale'] = coord['last3DaySale'].shift(1)
coord['last4DaySale'].fillna(method='bfill',inplace=True)
coord['last5DaySale'] = coord['last4DaySale'].shift(1)
coord['last5DaySale'].fillna(method='bfill',inplace=True)
coord['last6DaySale'] = coord['last5DaySale'].shift(1)
coord['last6DaySale'].fillna(method='bfill',inplace=True)
coord['last7DaySale'] = coord['last6DaySale'].shift(1)
coord['last7DaySale'].fillna(method='bfill',inplace=True)

coord['last8DaySale'] = coord['last7DaySale'].shift(1)
coord['last8DaySale'].fillna(method='bfill',inplace=True)
coord['last9DaySale'] = coord['last8DaySale'].shift(1)
coord['last9DaySale'].fillna(method='bfill',inplace=True)
coord['last10DaySale'] = coord['last9DaySale'].shift(1)
coord['last10DaySale'].fillna(method='bfill',inplace=True)
coord['last11DaySale'] = coord['last10DaySale'].shift(1)
coord['last11DaySale'].fillna(method='bfill',inplace=True)
coord['last12DaySale'] = coord['last11DaySale'].shift(1)
coord['last12DaySale'].fillna(method='bfill',inplace=True)
coord['last13DaySale'] = coord['last12DaySale'].shift(1)
coord['last13DaySale'].fillna(method='bfill',inplace=True)
coord['last14DaySale'] = coord['last13DaySale'].shift(1)
coord['last14DaySale'].fillna(method='bfill',inplace=True)



coord.loc[:,'diff2DaySale'] = coord['last1DaySale'] - coord['last3DaySale']
coord.loc[:,'diff3DaySale'] = coord['last1DaySale'] - coord['last4DaySale']
coord.loc[:,'diff4DaySale'] = coord['last1DaySale'] - coord['last5DaySale']
coord.loc[:,'diff5DaySale'] = coord['last1DaySale'] - coord['last6DaySale']
coord.loc[:,'diff6DaySale'] = coord['last1DaySale'] - coord['last7DaySale']
coord.loc[:,'diff7DaySale'] = coord['last1DaySale'] - coord['last8DaySale']

#1，2阶差分
coord.loc[:,'diff_1'] = coord['last1DaySale'] - coord.groupby('Class')['last1DaySale'].shift(1)
coord.loc[:,'diff_1'].fillna(0,inplace=True)
coord.loc[:,'diff_2'] = coord['diff_1'] - coord.groupby('Class')['diff_1'].shift(1)
coord.loc[:,'diff_2'].fillna(0,inplace=True)


coord.loc[:,'lastWeekTotSale'] = coord['last1DaySale'] + coord['last2DaySale'] + coord['last3DaySale'] + coord['last4DaySale'] + \
                                coord['last5DaySale'] + coord['last6DaySale'] + coord['last7DaySale']
coord.loc[:,'lastWeekSaleRate'] = coord['last1DaySale'] / (1.0 * coord['lastWeekTotSale'] + 0.1)

coord.loc[:,'daySale_pct'] = coord.groupby('Class')['last1DaySale'].pct_change(period=1)
coord.fillna(0,inplace=True)
coord['daySale_pct'][np.isnan(coord['daySale_pct'])] = 0

coord[coord['Class']==3236]

,Class,dayOfYear,saleCount,last1DaySale,last2DaySale,last3DaySale,last4DaySale,last5DaySale,last6DaySale,last7DaySale,...,diff3DaySale,diff4DaySale,diff5DaySale,diff6DaySale,diff7DaySale,diff_1,diff_2,lastWeekTotSale,lastWeekSaleRate,daySale_pct


In [32]:
lastWeeksSaleCount_mean = train_new.groupby(['Class','weekOfYear'],as_index=False)['saleCount'].agg({'WeekSaleCount_mean':'mean'})
lastWeeksSaleCount_mean['lastWeekSaleCount_mean']   = lastWeeksSaleCount_mean['WeekSaleCount_mean'].shift(1)
lastWeeksSaleCount_mean['lastWeekSaleCount_mean'].fillna(method='bfill',inplace=True)
lastWeeksSaleCount_mean

,Class,weekOfYear,WeekSaleCount_mean,lastWeekSaleCount_mean
0,10,1,7.500000,7.500000
1,10,2,12.428571,7.500000
2,10,3,10.285714,12.428571
3,10,4,12.428571,10.285714
4,10,5,15.428571,12.428571
5,10,6,7.000000,15.428571
6,10,7,15.285714,7.000000
7,10,8,7.428571,15.285714
8,10,9,3.714286,7.428571
9,10,10,9.571429,3.714286
